# Evaluación de la segmentación

In [1]:
import cv2 as cv
from matplotlib import pyplot as plt

In [2]:
image_10c = cv.imread('im/10c.jpg', cv.IMREAD_GRAYSCALE)
image_10t = cv.imread('im/10t.jpg', cv.IMREAD_GRAYSCALE)
image_all = cv.imread('im/all.jpg', cv.IMREAD_GRAYSCALE)
image_rgb = cv.imread('im/rgb.png', cv.IMREAD_GRAYSCALE)

In [3]:
im = image_10c
# Los números son las intensidades min y máx para identificar el borde
canny = cv.Canny(im, 25, 100)
cv.imwrite('out/canny.jpg', canny)

True

In [4]:
contornos, canny = cv.findContours(canny, cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
out = cv.drawContours(im, contornos, -1, (210,139,38), 2)
print('Número de contornos encontrados: ', len(contornos))
texto = 'Contornos encontrados: '+ str(len(contornos))
cv.putText(out, texto, (10,20), cv.FONT_HERSHEY_SIMPLEX, 0.7,
  (0, 0, 0), 1)
out = cv.cvtColor(out, cv.COLOR_BGR2RGB)
cv.imwrite('out/out.jpg', out)

Número de contornos encontrados:  16


True

# Otro

In [5]:
im = image_10t
# Imagen en binario
thresh = 128 #La mitad de bits
im_bin = cv.threshold(im, thresh, 255, cv.THRESH_BINARY)[1]

In [6]:
contornos, im_bin = cv.findContours(im_bin, cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
font = cv.FONT_HERSHEY_SIMPLEX
i=0
for c in contornos:
    M=cv.moments(c)
    if (M["m00"]==0): M["m00"]=1
    x=int(M["m10"]/M["m00"])
    y=int(M['m01']/M['m00'])

    mensaje = 'Num :' + str(i+1)
    cv.putText(im,mensaje,(x-40,y),font,0.75,(255,0,0),2,cv2.LINE_AA)
    cv.drawContours(im, [c], 0, (255,0,0),2)
  cv2.imshow('Imagen', imagen)
  cv2.waitKey(0)
  i = i+1
cv.imwrite('out/outupt.jpg', im_bin)

True